In [30]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle
import datetime

In [31]:
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

typeEchantillon : 
- fixe pour utiliser tailleEchantillon
- variable pour utiliser nombre,increment,nombreInitial
- total pour utiliser tout le lexique

In [32]:
typeEchantillon="fixe"
tailleEchantillon=30000
nombre=1000
increment=250
nombreInitial=20
freqForme=1000000
if typeEchantillon=="total":
    echantillonPrefix="-total"
elif typeEchantillon=="fixe":
    echantillonPrefix="-%d-%s" % (tailleEchantillon,dateheure())
else:
    echantillonPrefix="XXX"

In [33]:
filePrefix="MGC-150812"

##Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [34]:
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False

###Préparation des matrices de traits

features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')
#fs.supremum.concept.extent

bdlexiqueIn = unicode(u"èò")
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO")
neutralise = dict(zip(bdlexiqueNum, neutreOut))

def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    return result

###Définition des cases
- principales pour le paradigme courant
- secondaires pour les cases rares
- totales pour le tout

In [35]:
casesPrincipales= [
        'inf', 'pi1S', 'pi2S', 'pi3S', 'pi1P', 'pi2P', 'pi3P', 'ii1S',
        'ii2S', 'ii3S', 'ii1P', 'ii2P', 'ii3P', 
        'fi1S', 'fi2S', 'fi3S', 'fi1P', 'fi2P',
        'fi3P', 'pI2S', 'pI1P', 'pI2P', 'ps1S', 'ps2S', 'ps3S', 'ps1P',
        'ps2P', 'ps3P', 
        'pc1S', 'pc2S', 'pc3S', 'pc1P', 'pc2P', 'pc3P', 'pP',
        'ppMS', 'ppMP', 'ppFS', 'ppFP'
            ]
casesSecondaires= [
       'ai1S', 'ai2S', 'ai3S', 'ai1P', 'ai2P', 'ai3P', 'is1S', 'is2S', 'is3S', 'is1P', 'is2P', 'is3P'
            ]
casesTotales=casesPrincipales+casesSecondaires
listeCases=casesTotales

In [36]:
with open('MGC-150812-Verbes.pkl', 'rb') as input:
    verbes = pickle.load(input)

verbes["phono"]=verbes["phono"].apply(lambda x: recoder(x))

#Echantillonage

##Assembler les échantillons correspondant à une étape n

In [37]:
def assemblerExtrait(nombre):
    return verbes.ix[np.sort(np.concatenate(tirages[0:nombre]))]

###Tirage incrémental des formes disponibles
- increment : taille de chaque tirage
- nombre : nombre de tirages

np.random.choice donne une liste d'index de formes tirées dans l'ordre du tirage

on découpe la liste en morceaux de la taille de l'incrément

In [38]:
if typeEchantillon=="variable":
    tirage=np.random.choice(verbes.index,size=nombre*increment,p=verbes["prob"],replace=False)
    tirages=[tirage[increment*x:increment*(x+1)] for x in range(len(tirage)/increment+(len(tirage)%increment!=0))]
    extrait=assemblerExtrait(nombreInitial)
elif typeEchantillon=="fixe":
    tirage=np.random.choice(verbes.index,size=tailleEchantillon,p=verbes["prob"],replace=False)
    extrait=verbes.ix[np.sort(tirage)]
elif typeEchantillon=="total":
    extrait=verbes

In [39]:
paradigmes=pd.pivot_table(extrait, values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()

In [40]:
paradigmes.to_csv(path_or_buf=filePrefix+echantillonPrefix+"-paradigmes.csv",encoding="utf8",sep=";")

In [41]:
#with open(filePrefix+echantillonPrefix+'-Paradigmes.pkl', 'wb') as output:
#   pickle.dump(paradigmes, output, pickle.HIGHEST_PROTOCOL)

In [42]:
paradigmes

case,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ppFP,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
0,abaisser,NaN,NaN,NaN,NaN,abèsEr,abèsa,NaN,abès6rè,NaN,...,NaN,abèse,NaN,abèse,NaN,NaN,NaN,NaN,NaN,NaN
1,abandonner,NaN,abâdònè,NaN,NaN,NaN,abâdòna,abâdòn6rô,abâdòn6rè,NaN,...,abâdòne,abâdòne,abâdòne,abâdòne,NaN,NaN,NaN,abâdòn,abâdòn,abâdòn
2,abasourdir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,abazurdi,NaN,NaN,NaN,NaN,NaN,NaN
3,abattre,NaN,abati,NaN,NaN,abatir,abati,NaN,abatrè,NaN,...,NaN,abaty,abaty,abaty,NaN,abat,NaN,NaN,NaN,NaN
4,abdiquer,NaN,abdikè,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,abhorrer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,abjurer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,abolir,NaN,NaN,NaN,NaN,NaN,abòli,NaN,NaN,NaN,...,NaN,abòli,abòli,abòli,NaN,NaN,NaN,NaN,NaN,NaN
8,abonder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,abonner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
extrait[extrait["vs"]=="pp"]

,ortho,phono,ext,cs,ms,vs,lexeme,freq,prob,case
101,abaissé,abèse,NaN,K,MS,pp,abaisser,209000000,3.877337e-06,ppMS
102,abaissée,abèse,NaN,K,FS,pp,abaisser,44000000,8.162815e-07,ppFS
164,abandonné,abâdòne,NaN,K,MS,pp,abandonner,4648000000,8.622901e-05,ppMS
166,abandonnée,abâdòne,NaN,K,FS,pp,abandonner,1700000000,3.153815e-05,ppFS
168,abandonnées,abâdòne,"z""",K,FP,pp,abandonner,397000000,7.365085e-06,ppFP
170,abandonnés,abâdòne,"z""",K,MP,pp,abandonner,848000000,1.573197e-05,ppMP
177,abasourdi,abazurdi,NaN,K,MS,pp,abasourdir,244000000,4.526652e-06,ppMS
305,abattu,abaty,NaN,K,MS,pp,abattre,1891000000,3.508155e-05,ppMS
307,abattue,abaty,NaN,K,FS,pp,abattre,462000000,8.570955e-06,ppFS
313,abattus,abaty,"z""",K,MP,pp,abattre,485000000,8.997648e-06,ppMP
